In [ ]:
%matplotlib inline
import os, sys
import datetime as dt 
import yaml
import pandas as pd
import pcse
import pickle
from pcse.models import Wofost72_PP
from pcse.base import ParameterProvider
from pcse.exceptions import WeatherDataProviderError
from pcse.fileinput import ExcelWeatherDataProvider
import matplotlib.pyplot as plt


print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

# setting font and size 
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 16}
# change font
plt.rc('font', **font)

In [ ]:
# IND_10yrs = pd.DataFrame(wdp.export())
# IND_10yrs
# firstY = IND_10yrs[(IND_10yrs['DAY'] > dt.date(2013, 11, 1))& (IND_10yrs['DAY'] < dt.date(2014, 3, 1))]
# OUTPUT TO THE OUTSIDE FOR PROCESS
# firstY.to_excel('firstY_ind.xlsx')

In [ ]:
# prepared all artificial weather data in different periods
# rule is increase the TMAX by 5 degree in six different period 
# the interval for each period is 20 days 
import glob
wdps = []

file_paths = glob.glob(f"Period*.xlsx")
for file_path in file_paths:
    df = ExcelWeatherDataProvider(file_path)
    wdps.append(df)




In [ ]:
# load two data import functions from the pcse.fileinput class
from pcse.fileinput import YAMLCropDataProvider, CABOFileReader
# load the available crop list  
cropd = YAMLCropDataProvider(fpath='../data',
                              force_reload=True)  
# load the soil information
soild = CABOFileReader('../data/ec3 sandyloam.soil')
# load one data import function from the pcse.util class
from pcse.util import WOFOST72SiteDataProvider
# define the site initial condiations 
# WAV is the initial soil water content = 0; CO2 is the level of CO2 in the atmosphere = 360 ppm
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)
# help(WOFOST72SiteDataProvider)


In [ ]:
# Loop over crops, soils and years
agropotato = """
- 2013-11-02:
    CropCalendar:
        crop_name: potato
        variety_name: {Cultivar}
        crop_start_date: 2013-11-02
        crop_start_type: sowing
        crop_end_date: 2014-02-28
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null
"""
cultivars = [
    "Fontane", 
    "Markies",
    "Premiere",
    "Festien", 
    "Innovator"]

In [ ]:
res = []
for index, wdp in enumerate(wdps):
    for c in cultivars:
        parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
        agromanagement = yaml.safe_load(agropotato.format(Cultivar=c))
        wofost = Wofost72_PP(parameters, wdp, agromanagement)
        wofost.run_till_terminate()
        output = wofost.get_output()
        df = pd.DataFrame(output)
        df['cultivar'] = c
        df['period'] = f'Period{index}'
        res.append(df)

In [ ]:
all_data = pd.concat(res)
results = pd.concat(res)
results['day'] = pd.to_datetime(results['day'])
results.set_index('day', inplace=True)


In [ ]:
PeriodCate = ['Period' + str(i) for i in range(0,8)]

In [ ]:
# Create subplots
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

for ax, cultivar in zip(axes.flatten(), cultivars):
    df_period = results[results['cultivar'] == cultivar]
    for period in PeriodCate:
        df_cultivar = df_period[df_period['period'] == period]
        ax.plot(df_cultivar['LAI'].reset_index(drop=True), label=period)
    # ax.legend()
    ax.set_title(f"Cultivar {cultivar}")
axes[0][0].legend(['Normal', '1-20', '21-40','41-60','61-80','81-100','101-119'] ,loc='upper left')
fig.suptitle('The effect of manually adding 5 degree to TMAX on LAI over 20 days interval')

plt.tight_layout()
plt.show()

In [ ]:
# Create subplots
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

for ax, cultivar in zip(axes.flatten(), cultivars):
    df_period = results[results['cultivar'] == cultivar]
    for period in PeriodCate:
        df_cultivar = df_period[df_period['period'] == period]
        ax.plot(df_cultivar['TWSO'].reset_index(drop=True), label=period)
    # ax.legend()
    ax.set_title(f"Cultivar {cultivar}")
axes[0][0].legend(['Normal', '1-20', '21-40','41-60','61-80','81-100','101-119'] ,loc='upper left')
fig.suptitle('The effect of manually adding 5 degree to TMAX on TWSO over 20 days interval')

plt.tight_layout()
plt.show()

In [ ]:
6.5/0.2